In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd
from utils import create_driver


In [ ]:
main_url="https://propertyengine.co.uk/properties?_0:(area:%27!!UUvK78GuGmouNr%27,filters:!((options:(hideNewBuild:!f,hideOwnershipScheme:!t,hideRetirement:!t,hideSoldSTCRemoved:!t),type:dontShow),(options:(range:!(110000,490000)),type:askingPrice),(options:(selectedTypes:!(HOUSE)),type:propertyType),(options:(range:!(2,4)),type:bedrooms),(options:(filter:!f),type:auction),(options:(range:!(%272023-02-25%27,now)),type:addedDate)),sort:newest)"


browser=create_driver()
all_urls=["https://web.facebook.com/?_rdc=1&_rdr","https://www.youtube.com/"]
for url in all_urls:
    browser.execute_script('window.open("{}", "_blank");'.format(url))

In [ ]:
browser=create_driver()
# Navigate to a website in the existing Chrome instance
#main_url="https://propertyengine.co.uk/properties?_0:(area:ilford-greater-london,filters:!((options:(hideNewBuild:!f,hideOwnershipScheme:!t,hideRetirement:!t,hideSoldSTCRemoved:!t),type:dontShow),(options:(range:!(40000,!n)),type:askingPrice),(options:(range:!(3,4)),type:bedrooms)),sort:lowestPrice)"
#number_scrolls=30
browser.get(main_url)
wait = WebDriverWait(browser, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "*")))
time.sleep(10)
html_content = browser.page_source
soup = BeautifulSoup(html_content, 'html.parser')

count = int(soup.find("div" ,{"class": "sc-hBxehG fyjDiA"}).find_all("span")[0].text.split(" ")[2].replace(",",""))
print(count)
#zoom the page
#browser.execute_script("document.body.style.zoom='75%'")#150

recent_list = browser.find_element(By.XPATH, "//div[@class='sc-6jdl74-3 gjtvLs']")
number_scrolls=int(count/2)+1
# Scroll down 10 times within the div element
all_urls=[]
for i in tqdm(range(number_scrolls)):
    if i!=0:
        browser.execute_script("arguments[0].scrollTop += arguments[0].offsetHeight;", recent_list)
        time.sleep(5)#5
    html_content = browser.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    mydivs = soup.find_all("a" ,{"class": "f1gc2u-0 zpUJr"})

    for element in mydivs:
        #print(element["href"])
        #print(element)
        all_urls.append(element["href"])
    all_urls=list(set(all_urls))
    print(len(all_urls))
    if len(all_urls)>count:
        break
browser.quit()

#all_urls=list(set(all_urls))
print(len(all_urls))

In [2]:
def process_tab(driver, url):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(f'https://propertyengine.co.uk{url}')
    time.sleep(2)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    mydivs1 = soup.find_all("div" ,{"class": "q3lrp5-3 cLGsWF"})
    asking_price_text=mydivs1[0].text[:]
    mydivs3 = soup.find_all("div" ,{"class": "sc-gikAfH cnigbh"})
    print(url)
    if len(mydivs3)!=0 and len(mydivs1)!=0:
        #try:

        if asking_price_text.count("£")==2:
            asking_price_text=asking_price_text.split("£")[2]
        else:
            asking_price_text=asking_price_text[1:]

        asking_price=int(asking_price_text.replace(',', ''))
        average_price=int(mydivs3[0].text[1:].replace(',', ''))
        print(asking_price,average_price)


def scrape_urls(urls):
    driver = create_driver()
    for url in urls: 
        driver.execute_script("window.open('https://propertyengine.co.uk" + url + "');")
        time.sleep(0.5)
    time.sleep(20)
    driver.switch_to.window(driver.window_handles[-1])
    for handle in driver.window_handles[:-1]:
        time.sleep(3)
        driver.switch_to.window(handle)
        driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    for url in urls:
        process_tab(driver, url)
    driver.quit()

In [2]:
all_urls=pd.read_csv("all_urls.csv")
all_urls=list(all_urls["0"])
len(all_urls)

2016

In [3]:
main_results=[]

In [4]:
def process_tab(browser):
    html_content = browser.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    mydivs1 = soup.find_all("div" ,{"class": "q3lrp5-3 cLGsWF"})
    asking_price_text=mydivs1[0].text[:]
    mydivs3 = soup.find_all("div" ,{"class": "sc-gikAfH cnigbh"})
    if len(mydivs3)!=0 and len(mydivs1)!=0:
        #try:

        if asking_price_text.count("£")==2:
            asking_price_text=asking_price_text.split("£")[2]
        else:
            asking_price_text=asking_price_text[1:]

        asking_price=int(asking_price_text.replace(',', ''))
        average_price=int(mydivs3[0].text[1:].replace(',', ''))
        #print(asking_price,average_price)
        return [asking_price,average_price]
    else:
        return False

def scrape_urls(urls):
    url_info=[]
    url_chunks = [urls[i:i+50] for i in range(0, len(urls), 50)]
    print(len(url_chunks))
    count=0
    for chunk in url_chunks:
        print(count)
        driver = create_driver()
        info={"handles":[],"urls":[]}
        #handles = []
        for url in chunk:
            url_path=f'https://propertyengine.co.uk{url}'
            driver.execute_script("window.open('{}');".format(url_path))
            info["handles"].append(driver.window_handles[-1])
            #driver.switch_to.window(info["handles"][-1])
            #url_path=f'https://propertyengine.co.uk{url}'
            driver.get(url_path)
            info["urls"].append(url_path)
        all_handles=info["handles"]
        all_urls=info["urls"]
        for index in range(len(all_handles)):
        
            driver.switch_to.window(all_handles[index])
            try:
                results=process_tab(driver)
                if results==False:
                    results=[0,0]
                bye={"url":all_urls[index],"asking_price":results[0],"average_price":results[1]}
                if results[1]-results[0]>75000:
                    print(all_urls[index])
                    print(results[0],results[1])
                url_info.append(bye)
            except:
                pass
            driver.close()
        driver.quit()
        time.sleep(10)
        count+=1
    return url_info

In [5]:
all_results=scrape_urls(all_urls[500:1000])

10
0


In [ ]:
main_results.extend(all_results)

In [ ]:
len(main_results)

In [ ]:
browser=create_driver()
needed_url=[]
for link in tqdm(all_urls[:18]):
    url='https://propertyengine.co.uk{}'.format(link)
    browser.get('https://propertyengine.co.uk{}'.format(link))
    browser.execute_script("window.scrollBy(0,500)")
    #browser.execute_script("document.body.style.zoom='150%'")
    time.sleep(5)
    html_content = browser.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    mydivs1 = soup.find_all("div" ,{"class": "q3lrp5-3 cLGsWF"})
    asking_price_text=mydivs1[0].text[:]
    mydivs3 = soup.find_all("div" ,{"class": "sc-gikAfH cnigbh"})
    print(url)
    if len(mydivs3)!=0 and len(mydivs1)!=0:
        #try:

        if asking_price_text.count("£")==2:
            asking_price_text=asking_price_text.split("£")[2]
        else:
            asking_price_text=asking_price_text[1:]

        asking_price=int(asking_price_text.replace(',', ''))
        average_price=int(mydivs3[0].text[1:].replace(',', ''))
        #print(url)
        print(asking_price,average_price)
        if average_price-asking_price>50000:#
            needed_url.append(url)
        # except:
        #     pass
    # Quit the browser
browser.quit()


In [ ]:
print(len(main_results))
import pandas as pd
df = pd.DataFrame(main_results)
#df.to_csv("csv_file.csv")

In [ ]:
df2=df.loc[df["average_price"]-df["asking_price"]>75000]
len(df2)

In [ ]:
df2.to_csv("first_500_refined.csv",index=False)